In [1]:
import pandas as pd
import numpy as np
input_excel_file = "/home/chirag/Desktop/snippet.xlsx"

In [2]:
def agent_customer(input_excel_file):    
    df = pd.read_excel(input_excel_file)
    df.text= df.text.astype(str)
    df['a_bin'] = 0
    df['b_bin'] = 0
    df.a_bin = df.speaker.apply(lambda x: 0 if x=='Agent' else 1)
    df.b_bin = df.speaker.apply(lambda x: 0 if x=='Customer' else 1)
    df['a_bin_cumsum'] = df.a_bin.cumsum()
    df['b_bin_cumsum'] = df.b_bin.cumsum()
    df = df.drop(['a_bin','b_bin'],axis=1)
    df['a_bin'] = df.speaker.apply(lambda x: 1 if x=='Agent' else 0)
    df['b_bin'] = df.speaker.apply(lambda x: 1 if x=='Customer' else 0)
    df['a_con'] = df.a_bin_cumsum*df.a_bin 
    df['b_con'] = df.b_bin_cumsum*df.b_bin
    df.drop(['a_bin_cumsum','b_bin_cumsum','a_bin','b_bin'],axis=1,inplace=True)
    df['identifier'] = df.a_con + df.b_con
    df['name_idnet'] = df.speaker+"_"+df.identifier.astype(str)
    df.drop(['a_con','b_con','identifier'],axis=1,inplace=True)
    df['nindex'] = df.index

    def summing(df):
        df = df.copy().reset_index(drop=True)
        n_ind = df.nindex[0].astype(int)
        if df.shape[0] == 1:
            df['nindx'] = n_ind
            return(df.reset_index(drop=True))
        else:
            df['text'] = df.text.astype(str) + " "
            df['id'] = df.id.astype(str)+ " "
            text = df.text.sum().strip()
            name = df.name_idnet[0]
            from_time = df.from_time[0]
            to_time = df.to_time[df.shape[0]-1]
            ids = df.id.sum().strip().replace(" ",",")
            data = {'nindx':[n_ind],'id':[ids],'speaker':[df.speaker[0]],'name_idnet':[df.name_idnet[0]],
                    'text':[text],'from_time':[from_time],'to_time':[to_time]}
            df1 = pd.DataFrame(data)        
            return(df1.reset_index(drop=True))

    df_result = df.groupby(['speaker','name_idnet'],as_index=False).apply(summing).drop('nindex',axis=1).sort_values('nindx').reset_index(drop=True)
    df_result = df_result.drop(['name_idnet', 'nindx'], axis=1)
    df_result = df_result[['id', 'speaker', 'text', 'from_time', 'to_time']]
    return df_result

In [3]:
input_excel_file = "/home/chirag/Desktop/snippet.xlsx"
df = agent_customer(input_excel_file)

/home/chirag/venv/lib/python3.6/site-packages/pandas/core/groupby/groupby.py:924: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  result = concat(values, axis=self.axis, keys=keys)
